## APRENDIZAJE INCREMENTAL DIARIO - MODELO DE DETECCIÓN DE FRAUDE

### -Este notebook actualiza el modelo general con datos del día anterior
### -Se ejecuta una vez al día para mantener el modelo actualizado

In [1]:
import pandas as pd
import numpy as np
from river import anomaly
from river import preprocessing
import pickle
import os
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("✅ Librerías importadas para aprendizaje incremental")

✅ Librerías importadas para aprendizaje incremental


# # 1. CONFIGURACIÓN Y RUTAS

In [2]:
# Rutas de archivos
MODELS_PATH = r"C:\Users\User\Desktop\TESIS\CodigoGithub\MLTallerProyecto1\Modelos"
DAILY_DATA_PATH = r"C:\Users\User\Desktop\TESIS\Datos_Diarios"  # Aquí llegan los CSVs diarios

# Crear directorio de datos diarios si no existe
os.makedirs(DAILY_DATA_PATH, exist_ok=True)

# Fecha del día anterior (para el archivo a procesar)
archivo_ayer = "datos_primer_dia.csv"
ruta_archivo_ayer = os.path.join(DAILY_DATA_PATH, archivo_ayer)

print(f"📁 Directorio de modelos: {MODELS_PATH}")
print(f"📁 Directorio de datos diarios: {DAILY_DATA_PATH}")
print(f"📄 Archivo a procesar: {archivo_ayer}")

📁 Directorio de modelos: C:\Users\User\Desktop\TESIS\CodigoGithub\MLTallerProyecto1\Modelos
📁 Directorio de datos diarios: C:\Users\User\Desktop\TESIS\Datos_Diarios
📄 Archivo a procesar: datos_primer_dia.csv


# # 2. CARGAR MODELO Y CONFIGURACIÓN EXISTENTE

In [4]:
modelo_path = os.path.join(MODELS_PATH, "modelo_general.pkl")
with open(modelo_path, 'rb') as f:
    modelo = pickle.load(f)

# Cargar scaler
scaler_path = os.path.join(MODELS_PATH, "scaler_general.pkl")
with open(scaler_path, 'rb') as f:
    scaler = pickle.load(f)

# Cargar configuración
config_path = os.path.join(MODELS_PATH, "config_modelo_general.pkl")
with open(config_path, 'rb') as f:
    config = pickle.load(f)

# Extraer componentes importantes
stats_dict = config['stats_por_pais']
contexto_historico = config['contexto_historico']  # CONTEXTOS PERMANENTES
umbral_global = config['umbral_global']
parametros_features = config['parametros_features']

print(f"✅ Modelo cargado exitosamente")
print(f"📊 Países en contexto histórico: {len(contexto_historico)}")
print(f"📊 Países en stats actuales: {len(stats_dict)}")
print(f"🎯 Umbral actual: {umbral_global:.4f}")

✅ Modelo cargado exitosamente
📊 Países en contexto histórico: 188
📊 Países en stats actuales: 188
🎯 Umbral actual: 0.9724


# # 3. VERIFICAR Y CARGAR DATOS DEL DÍA ANTERIOR


In [ ]:
print(f"\n📄 Verificando datos del día anterior...")

if not os.path.exists(ruta_archivo_ayer):
    print(f"❌ ERROR: No se encontró el archivo {archivo_ayer}")
    print(f"📁 Ruta esperada: {ruta_archivo_ayer}")
    print("🔍 Archivos disponibles en el directorio:")
    
    if os.path.exists(DAILY_DATA_PATH):
        archivos = [f for f in os.listdir(DAILY_DATA_PATH) if f.endswith('.csv')]
        for archivo in archivos:
            print(f"   - {archivo}")
    else:
        print("   (Directorio no existe)")
    
    # Usar archivo más reciente si existe
    if os.path.exists(DAILY_DATA_PATH):
        archivos = [f for f in os.listdir(DAILY_DATA_PATH) if f.endswith('.csv')]
        if archivos:
            archivo_mas_reciente = max(archivos)
            ruta_archivo_ayer = os.path.join(DAILY_DATA_PATH, archivo_mas_reciente)
            print(f"📄 Usando archivo más reciente: {archivo_mas_reciente}")
        else:
            raise FileNotFoundError("No hay archivos CSV disponibles para procesar")
    else:
        raise FileNotFoundError("Directorio de datos diarios no existe")

# Cargar datos del día anterior
print(f"🔄 Cargando datos de {os.path.basename(ruta_archivo_ayer)}...")
df_ayer = pd.read_csv(ruta_archivo_ayer)

# Convertir fecha si existe
if 'FECHA' in df_ayer.columns:
    df_ayer['FECHA'] = pd.to_datetime(df_ayer['FECHA'], format='%d/%m/%Y', errors='coerce')

print(f"📋 Datos cargados - Shape: {df_ayer.shape}")
print(f"🌍 Países en datos nuevos: {df_ayer['CODIGODEPAIS'].nunique()}")

# # 4. FUNCIÓN DE FEATURES (IMPORTADA DEL MODELO ORIGINAL)


In [ ]:
def crear_features_contextualizadas_mejorada(row, stats_pais_dict):
    """
    Función idéntica a la del modelo original - MANTENER CONSISTENCIA
    """
    pais = row['CODIGODEPAIS']
    llamadas = row['N_LLAMADAS']
    minutos = row['N_MINUTOS']
    destinos = row['N_DESTINOS']
    
    # Usar parámetros guardados en config
    PESO_MINUTOS_NORMAL = parametros_features['peso_minutos_normal']
    PESO_MINUTOS_EXTREMOS = parametros_features['peso_minutos_extremos']
    UMBRAL_MINUTOS_EXTREMOS = parametros_features['umbral_minutos_extremos']
    PESO_DESTINOS = parametros_features['peso_destinos']
    PESO_SPRAY_RATIO = parametros_features['peso_spray_ratio']
    
    # Obtener contexto del país (si existe)
    if pais in stats_pais_dict:
        pais_stats = stats_pais_dict[pais]
        categoria = pais_stats['CATEGORIA']
        
        # Normalizar por el contexto del país
        llamadas_norm = min(llamadas / max(pais_stats['LLAMADAS_P95'], 1), 1.5)
        destinos_norm = min(destinos / max(pais_stats['DESTINOS_P95'], 1), 1.5)
        
        # Detección inteligente de minutos extremos
        minutos_p90 = pais_stats.get('MINUTOS_P90', pais_stats['MINUTOS_P95'] * 0.9)
        minutos_p95 = pais_stats['MINUTOS_P95']
        
        # Transformación adaptativa de minutos
        if minutos >= UMBRAL_MINUTOS_EXTREMOS:
            minutos_norm = min(minutos / max(minutos_p90, 1), 3.0)
            peso_minutos = PESO_MINUTOS_EXTREMOS
        else:
            minutos_norm = min(np.log1p(minutos) / np.log1p(max(minutos_p90, 1)), 1.2)
            peso_minutos = PESO_MINUTOS_NORMAL
            
    else:
        # País nuevo - SIEMPRE clasificar como 'Muy_Bajo'
        categoria = 'Muy_Bajo'
        llamadas_norm = min(llamadas / 10, 2.0)
        destinos_norm = min(destinos / 5, 2.0)
        
        if minutos >= UMBRAL_MINUTOS_EXTREMOS:
            minutos_norm = min(minutos / 50, 3.0)
            peso_minutos = PESO_MINUTOS_EXTREMOS * 1.2
        else:
            minutos_norm = min(np.log1p(minutos) / np.log1p(60), 1.2)
            peso_minutos = PESO_MINUTOS_NORMAL
    
    # Features principales (idénticas al modelo original)
    features = {
        'llamadas_norm': llamadas_norm * 0.8,
        'destinos_norm': destinos_norm * PESO_DESTINOS,
        'minutos_norm': minutos_norm * peso_minutos,
        'diversidad_destinos': min(destinos / max(llamadas, 1), 1.0),
        'spray_ratio': min(destinos / max(llamadas, 1) * PESO_SPRAY_RATIO, 1.0) if destinos >= 5 else 0,
        'minutos_extremos': 1.0 if minutos >= UMBRAL_MINUTOS_EXTREMOS else 0.0,
        'minutos_sospechosos': min((minutos - 200) / 300, 1.0) if minutos > 200 else 0.0,
        'patron_spray_fuerte': 1.0 if (destinos >= 10 and llamadas >= 20) else 0.0,
        'patron_spray_medio': 0.5 if (destinos >= 6 and llamadas >= 12) else 0.0,
        'alta_diversidad': min(destinos / 12, 1) if destinos >= 5 else 0,
        'volumen_llamadas_alto': min((llamadas - 30) / 50, 1) if llamadas > 30 else 0,
        'volumen_destinos_alto': min((destinos - 10) / 20, 1) if destinos > 10 else 0,
        'llamadas_por_destino': min(llamadas / max(destinos, 1) / 5, 1),
        'eficiencia_destinos': min(destinos / max(llamadas * 0.5, 1), 1),
        'factor_pais_bajo': 1.5 if categoria in ['Muy_Bajo', 'Bajo'] else 1.0,
        'factor_pais_alto': 0.9 if categoria in ['Alto', 'Medio'] else 1.0
    }
    
    return features

print("🔧 Función de features cargada (consistente con modelo original)")

# # 5. APRENDIZAJE INCREMENTAL


In [ ]:
print(f"\n🎓 Iniciando aprendizaje incremental...")

# Contadores
registros_procesados = 0
paises_nuevos = set()
registros_por_pais = {}

# Procesar cada registro del día anterior
for idx, row in df_ayer.iterrows():
    # Crear features
    features = crear_features_contextualizadas_mejorada(row, stats_dict)
    
    # Normalizar con el scaler existente
    scaler.learn_one(features)  # Actualizar scaler incrementalmente
    features_scaled = scaler.transform_one(features)
    
    # Obtener score antes del aprendizaje (para logging)
    score_antes = modelo.score_one(features_scaled)
    
    # APRENDER del nuevo registro
    modelo.learn_one(features_scaled)
    
    # Estadísticas
    pais = row['CODIGODEPAIS']
    registros_procesados += 1
    
    # Detectar países nuevos
    if pais not in contexto_historico and pais not in stats_dict:
        paises_nuevos.add(pais)
    
    # Contar registros por país
    registros_por_pais[pais] = registros_por_pais.get(pais, 0) + 1
    
    if registros_procesados % 1000 == 0:
        print(f"   Procesados: {registros_procesados}/{len(df_ayer)} registros")

print(f"✅ Aprendizaje incremental completado")
print(f"📊 Registros procesados: {registros_procesados}")
print(f"🆕 Países nuevos detectados: {len(paises_nuevos)}")

if paises_nuevos:
    print(f"🌍 Países nuevos: {list(paises_nuevos)[:10]}...")  # Mostrar primeros 10

# # 6. ACTUALIZAR ESTADÍSTICAS POR PAÍS


In [ ]:
print(f"\n📊 Actualizando estadísticas por país...")

# Calcular nuevas estadísticas para países con suficientes datos
stats_nuevos = {}
umbral_minimo = 20  # Mínimo de registros para actualizar stats

for pais in df_ayer['CODIGODEPAIS'].unique():
    datos_pais = df_ayer[df_ayer['CODIGODEPAIS'] == pais]
    
    if len(datos_pais) >= umbral_minimo:
        # Calcular estadísticas nuevas
        stats_nuevos[pais] = {
            'REGISTROS': len(datos_pais),
            'LLAMADAS_MEAN': datos_pais['N_LLAMADAS'].mean(),
            'LLAMADAS_STD': datos_pais['N_LLAMADAS'].std(),
            'LLAMADAS_P90': datos_pais['N_LLAMADAS'].quantile(0.90),
            'LLAMADAS_P95': datos_pais['N_LLAMADAS'].quantile(0.95),
            'MINUTOS_MEAN': datos_pais['N_MINUTOS'].mean(),
            'MINUTOS_STD': datos_pais['N_MINUTOS'].std(),
            'MINUTOS_P90': datos_pais['N_MINUTOS'].quantile(0.90),
            'MINUTOS_P95': datos_pais['N_MINUTOS'].quantile(0.95),
            'DESTINOS_MEAN': datos_pais['N_DESTINOS'].mean(),
            'DESTINOS_STD': datos_pais['N_DESTINOS'].std(),
            'DESTINOS_P90': datos_pais['N_DESTINOS'].quantile(0.90),
            'DESTINOS_P95': datos_pais['N_DESTINOS'].quantile(0.95),
        }
        
        # Clasificar país por volumen
        registros_totales = stats_nuevos[pais]['REGISTROS']
        if pais in stats_dict:
            registros_totales += stats_dict[pais].get('REGISTROS', 0)
        
        if registros_totales <= 50:
            categoria = 'Muy_Bajo'
        elif registros_totales <= 200:
            categoria = 'Bajo'
        elif registros_totales <= 1000:
            categoria = 'Medio'
        else:
            categoria = 'Alto'
        
        stats_nuevos[pais]['CATEGORIA'] = categoria

# Fusionar estadísticas existentes con nuevas
stats_actualizados = stats_dict.copy()

for pais, stats_nuevas in stats_nuevos.items():
    if pais in stats_actualizados:
        # Promediar con estadísticas existentes (weighted average)
        peso_existente = 0.7  # Dar más peso a datos históricos
        peso_nuevo = 0.3
        
        for campo in ['LLAMADAS_MEAN', 'MINUTOS_MEAN', 'DESTINOS_MEAN']:
            if campo in stats_actualizados[pais]:
                valor_existente = stats_actualizados[pais][campo]
                valor_nuevo = stats_nuevas[campo]
                stats_actualizados[pais][campo] = (valor_existente * peso_existente + 
                                                 valor_nuevo * peso_nuevo)
        
        # Actualizar percentiles con datos más recientes
        for campo in ['LLAMADAS_P90', 'LLAMADAS_P95', 'MINUTOS_P90', 'MINUTOS_P95', 
                     'DESTINOS_P90', 'DESTINOS_P95']:
            if campo in stats_nuevas:
                stats_actualizados[pais][campo] = stats_nuevas[campo]
        
        # Actualizar categoría si cambió
        stats_actualizados[pais]['CATEGORIA'] = stats_nuevas['CATEGORIA']
        
    else:
        # País nuevo - agregar completamente
        stats_actualizados[pais] = stats_nuevas
        
        # IMPORTANTE: Países nuevos NO se agregan al contexto histórico
        # El contexto histórico permanece INMUTABLE

print(f"📊 Estadísticas actualizadas para {len(stats_nuevos)} países")
print(f"📊 Total países en estadísticas: {len(stats_actualizados)}")
print(f"🔒 Contexto histórico INMUTABLE: {len(contexto_historico)} países")


# # 7. GUARDAR MODELO ACTUALIZADO


In [ ]:
print(f"\n💾 Guardando modelo actualizado...")

# Guardar modelo actualizado
modelo_path = os.path.join(MODELS_PATH, "modelo_general.pkl")
with open(modelo_path, 'wb') as f:
    pickle.dump(modelo, f)

# Guardar scaler actualizado
scaler_path = os.path.join(MODELS_PATH, "scaler_general.pkl")
with open(scaler_path, 'wb') as f:
    pickle.dump(scaler, f)

# Actualizar configuración
config['stats_por_pais'] = stats_actualizados
config['fecha_ultima_actualizacion'] = datetime.now().isoformat()
config['registros_ultima_actualizacion'] = registros_procesados
config['paises_nuevos_detectados'] = list(paises_nuevos)

# IMPORTANTE: contexto_historico NO se modifica
# config['contexto_historico'] permanece igual

# Guardar configuración actualizada
config_path = os.path.join(MODELS_PATH, "config_modelo_general.pkl")
with open(config_path, 'wb') as f:
    pickle.dump(config, f)

# Crear respaldo con timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
backup_path = os.path.join(MODELS_PATH, f"backup_config_{timestamp}.pkl")
with open(backup_path, 'wb') as f:
    pickle.dump(config, f)

print("✅ Modelo actualizado y guardado exitosamente")
print(f"📁 Backup creado: backup_config_{timestamp}.pkl")